## Injection benchmark

### Setup

CPython 3.7.2 and an i7 7700K were used for the timings.
The antidote version used

In [1]:
import sys
from antidote import __version__, is_compiled
print(f"Antidote: {__version__()} {'(cython)' if is_compiled() else ''}")
print(f"Python {sys.version}")

Antidote: 0.5.0 (cython)
Python 3.7.2 (default, Dec 29 2018, 19:09:49) 
[GCC 7.3.0]


### Results
TLDR: Impact is negligeable for most use cases.

The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the other cases, the overhead is considered to be negligeable.

It should be noted that in most cases the worst scenario is used, as functions do nothing. In the real world, pure python functions are a lot slower. So to put the following results into perspective, here is the time needed to decode this simple JSON.

In [2]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

4.34 µs ± 65.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
from antidote import world, register, inject

In [4]:
@register
class Service1:
    pass


@register
class Service2:
    def __init__(self, service1: Service1):
        self.service1 = service1


@register
class Service3:
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@register
class Service4:
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3


### Function call

Injection overhead is here measured with a function which does nothing.

In [5]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

Time necessary to only execute the function, without retrieving the services

In [6]:
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
%timeit f(*args)

243 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when all argument must be retrieved from the container.

In [7]:
f_injected = inject(f)
assert f(*args) == f_injected()
%timeit f_injected()

835 ns ± 3.86 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [8]:
assert f(*args) == f_injected(*args)
%timeit f_injected(*args)

327 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation

In [9]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

679 ns ± 5.87 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
@register
class ObjInjected:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()


1.79 µs ± 6.15 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Configuration


In [11]:
from antidote import LazyConfigurationMeta

class Conf(metaclass=LazyConfigurationMeta):
    A = 'A'
    B = 'B'

    def __call__(self, key):
        return key

In [12]:
def g(a, b):
    return a, b

%timeit g('A', 'B')

205 ns ± 0.336 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
conf = Conf()
%timeit g(conf('A'), conf('B'))

586 ns ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
g_injected = inject(g, dependencies=(Conf.A, Conf.B))

assert g(conf('A'), conf('B')) == g_injected()
assert g(conf.A, conf.B) == g_injected()

%timeit g_injected()

596 ns ± 6.58 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
